!git clone https://github.com/yoonkim313/dataCampusProject-Team10.git

    ! python3 /content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/ocr/deep-text-recognition/train.py
    ! python /content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/main.ipynb
    sys.path.insert(0,nb_path)
    !pip install --target=$nb_path transformers
    !apt-get update
    !apt-get g++ openjdk-8-jdk 
    !pip3 install --target=$nb_path konlpy
    !pip install --target=$nb_path soykeyword
    !pip install --target=$nb_path krwordrank
    !pip install --target=$nb_path heapq
    !pip install --target=$nb_path kss
    !pip install --target=$nb_path bert
    !pip install --target=$nb_path textrankr
    !pip install --target=$nb_path lexrankr
    %cd /content/drive/Shared drives/BigDATA TEAM 10/py-hanspell
    !python setup.py install

In [40]:
#@title A-Catcher 실행 환경 구축
import os, sys
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Shared drives/BigDATA TEAM 10
import warnings
warnings.filterwarnings('ignore')
from konlpy.tag import Hannanum, Okt
h = Hannanum()
o = Okt()
import re
import time
import numpy as np
import heapq
import pandas as pd
from operator import itemgetter
from collections import deque, defaultdict
from ast import literal_eval
from collections import defaultdict
from pprint import pprint
from krwordrank.word import KRWordRank
from copy import deepcopy
import kss
import itertools
import unicodedata
import requests
from functools import reduce
import torch
from bs4 import BeautifulSoup
import string
from textrankr import TextRank
from lexrankr import LexRank
from hanspell import spell_checker  

!pip install --upgrade google-cloud-vision
!apt-get install -y poppler-utils

%cd /content/drive/'Shared drives'/'BigDATA TEAM 10'
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request, redirect, url_for
from flask_admin.contrib.fileadmin import FileAdmin
from flask_admin import Admin
from flask_dropzone import Dropzone
from pdf2image import convert_from_path
import io
import os
from google.cloud import vision
from google.cloud.vision import types
import logging
from pptx import Presentation
from pptx_tools import utils


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shared drives/BigDATA TEAM 10
Requirement already up-to-date: google-cloud-vision in /usr/local/lib/python3.6/dist-packages (1.0.0)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.10).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
/content/drive/Shared drives/BigDATA TEAM 10


In [41]:
#@title 자연어 처리 모듈
class Text():
    def __init__(self, text):
        
        
        text = re.sub("'", ' ', text)
        paragraphs = text.split('\n')
        self.text = text
        self.paragraphs = [i for i in paragraphs if i]
        self.counts = len(self.paragraphs)
        self.docs = [kss.split_sentences(paragraph) for paragraph in paragraphs if kss.split_sentences(paragraph)]
        self.newtext = deepcopy(self.text)
        print("TEXT")

    def findall(self, p, s):
        i = s.find(p)
        while i != -1:
            yield i
            i = s.find(p, i + 1)
      
    def countMatcher(self, sentences, paragraph_no):
        paragraph = self.docs[paragraph_no]
        total_no = len(paragraph)
        vec = [0] * total_no
        
        for idx, candidate in enumerate(paragraph):
            for sentence in sentences:
                if sentence[:4] in candidate:
                    vec[idx] += 1
        print("Vec ",vec)
        return vec

class Highlight(Text):
    def __init__(self, text):
        super().__init__(text)

        print("Highlight")
        okt = Okt()
        h = Hannanum()

        wordrank_extractor = KRWordRank(min_count=4, max_length=10)
        self.keywords, rank, graph = wordrank_extractor.extract(self.paragraphs, num_keywords=20)
        self.path = "/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction"
        p = []
        kw = []
        for k, v in self.keywords.items():
            p.append(okt.pos(k))
            kw.append(k)
        words = self.text.split(' ')
        s = set()
        keylist = [word for i in kw for word in words if i in word]
        keylist = [i for i in keylist if len(i)>2]
        for i in keylist:
            if len(i)>2:
              s.add(i)
        print("KEYLIST: ",keylist)
        p = [okt.pos(word) for word in s]
        s = set()
        for idx in range(len(p)):
            ls = p[idx]
            for j in range(len(ls)):
                tag = ls[j][1]
                word = ls[j][0]
                if tag == "Noun":
                    s.add(word)
        self.keys = []
        for temp in s:
            self.keys.append(" " + temp)

    
    def add_tags_keywords(self):
        self.candidates = self.keys
        self.newtext = deepcopy(self.text)
        self.idx = [(i, i + len(candidate)) for candidate in self.candidates for i in
                        self.findall(candidate, self.text)]
        for i in range(len(self.idx)):
            try:
                self.idx = [(start, start + len(candidate)) for candidate in self.candidates for start in
                            self.findall(candidate, self.newtext)]
                word = self.newtext[self.idx[i][0]:self.idx[i][1]]
                tagged = " <mark style='background-color:#FFD0F2'>%s</mark>" % (word)
                self.newtext = tagged.join([self.newtext[:self.idx[i][0]], self.newtext[self.idx[i][1]:]])
            except:
                pass
        return self.newtext


    def add_tags_conj(self, txt):
        conj = '그리고, 그런데, 그러나, 그래도, 그래서, 또는, 및, 즉, 게다가, 따라서, 때문에, 아니면, 왜냐하면, 단, 오히려, 비록, 예를 들어, 반면에, 하지만, 그렇다면, 바로, 이에 대해'
        conj = conj.replace("'", "")
        self.candidates = conj.split(",")
        self.newtext = deepcopy(txt)
        self.idx = [(i, i + len(candidate)) for candidate in self.candidates for i in
                        self.findall(candidate, txt)]
        for i in range(len(self.idx)):
            try:
                self.idx = [(start, start + len(candidate)) for candidate in self.candidates for start in
                            self.findall(candidate, self.newtext)]
                word = self.newtext[self.idx[i][0]:self.idx[i][1]]
                res = ""
                tagged = " <mark style='background-color:#F9D877'>%s</mark>" % (word)
                print(tagged)
                self.newtext = tagged.join([self.newtext[:self.idx[i][0]], self.newtext[self.idx[i][1]:]])
            except:
                pass
        return self.newtext

    def highlight(self):
          result = self.add_tags_keywords()
          cleanr = re.compile("</mark>.{0,5}<mark style='background-color:#FFD0F2'>")
          patterns = re.findall("</mark>.{0,5}<mark style='background-color:#FFD0F2'>",result)
          w = [i[7:-39] for i in patterns]
          for i in range(len(w)):
            result = re.sub(patterns[i], w[i], result)
          txt = self.add_tags_conj(result)
          print("Highlighted Result ", txt)
          return txt

class Relation(Highlight):
    def __init__(self):
      super().__init__(text)
      
    def frameParse(self, id):
        parser = frame_parser.FrameParser(model_path=self.path, language='ko')
        ps = parser.parser(self.docs[id], sent_id='1', result_format='conll')
        return ps

    def extractFrame(self):
        self.final = {}
        for paragraph in self.docs:
            print("PARAGRAPH: ", self.docs)
            for idx in range(len(paragraph)):
                parsed = frameParse(paragraph[idx])  # candidates 생성
                self.final.setdefault(idx, str)
                parsedList = LinkedList()
                for j in range(len(parsed)):
                    parsed_candidate = parsed[j]
                    new_node = Node(parsed_candidate)
                    if j == 0:
                        old_node = new_node
                        parsedList.head = old_node
                    elif j == len(parsed) - 1:
                        old_node.next = new_node
                        new_node.next = None
                        print(idx, '  ', parsedList)
                        self.final[idx] = parsedList
                    else:
                        old_node.next = new_node
                        old_node = new_node

    def findConsecutiveBIO(self, words, tag):
        began = False
        count = 1
        self.que = deque(words)
        for i in range(len(words)):
            if tag[i] == 'O' and not began:
                self.que.popleft()
                began = False
            if tag[i] == 'O' and began:
                self.que.pop()
                began = False
            if tag[i].startswith('B'):
                began = True
            if began & tag[i].startswith('I'):
                count += 1
        return self.que

    def extractRelation(self):
        s = ""
        self.res = [] * 20
        for k, v in self.final.items():
            a = v.head
            try:
                while a is not None:
                    i = 0
                    temp = " "
                    if len(a.words) == len(a.tags):
                        q = findConsecutiveBIO(a.words, a.tags)
                        print(f"The que {q}")
                    else:
                        print("ERROR OCCURED")

                    count = len(a.words)
                    for tag in a.tags:
                        if tag.startswith("O"):
                            count -= 1
                    if count > len(a.words) * 0.45:
                        print(f"The threshold is reached !!😏 The count is {count}")
                        temp = " ".join(q)
                        words = a.words
                        tags = a.tags
                    else:
                        temp = None
                        words = None
                        tags = None
                    self.res[i] = (words, tag, temp)
                    i+=1
                    a = a.next
            except:
                pass
        self.res = [j for j in self.res if j]

    def roles(self):
        for k, tup in self.res.items():
            a, b, c = tup
            roles = [0] * len(b)
            words = [0] * len(b)
            for i in range(len(b)):
                try:
                    roles[i] = b[i].split("-")[1]
                except:
                    roles[i] = b[i].split("-")[0]
            print(roles)
            for _ in roles:
                pass

class Summarize(Highlight):
    def __init__(self, text, paragraph_no):
      super().__init__(text)
      print("length of paragraphs ",len(self.paragraphs))
      self.txt = self.paragraphs[paragraph_no]
      self.paragraph_no = paragraph_no

    def summarize(self):
        url = "https://api.smrzr.io/summarize?ratio=0.15"
        headers = {
            'content-type': 'raw/text',
            'origin': 'https://smrzr.io',
            'referer': 'https://smrzr.io/',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors',
            'sec-fetch-site': 'same-site',
            "user-agent": "Mozilla/5.0"
        }
        resp = requests.post(url, headers=headers, data= self.txt.encode('utf-8'))
        assert resp.status_code == 200
        summary = resp.json()['summary']
        temp = summary.split('\n')
        print("BERT: ", temp)
        return temp


    def summarizeTextRank(self, max=1):
        tr = TextRank(self.txt)
        
        summary = tr.summarize(max).split('\n')
        print("Textrank: ",summary)
        return summary


    def summarizeLexRank(self):
        lr = LexRank()
        lr.summarize(self.txt)
        summaries = lr.probe()
        print("Lexrank: ",summaries)
        return summaries

    def ensembleSummarize(self):
        a = np.array(self.countMatcher(self.summarize(), self.paragraph_no))
        
        try:
          b = np.array(self.countMatcher(self.summarizeLexRank(), self.paragraph_no))
        except:
          b = np.zeros_like(a)
        c = np.array(self.countMatcher(self.summarizeTextRank(),self.paragraph_no))
        result= a+b+c
        i, = np.where(result == max(result))
        txt, index = self.docs[self.paragraph_no][i[0]], i[0]
        return txt, index

####################################################################################################


import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/ocr/My Project-a3f9a30ceabf.json"
client = vision.ImageAnnotatorClient()

In [51]:
#@title Flask 작동 코드
%cd /content/drive/'Shared drives'/'BigDATA TEAM 10'/dataCampusProject-Team10/flask
app = Flask(__name__)
run_with_ngrok(app)

basedir = '/content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/flask' 
upload_dir = os.path.join(basedir, 'uploads')  # basedir 의 uploads 에 파일 저장
#################################################################################################################

admin = Admin(name='Uploaded Files')
admin.init_app(app)
dropzone = Dropzone(app)
admin.add_view(FileAdmin(upload_dir, name='FILES')) 
app.config['DROPZONE_ALLOWED_FILE_CUSTOM'] = True
app.config['DROPZONE_ALLOWED_FILE_TYPE'] = 'image/*, .pdf, .txt'
app.config['DROPZONE_REDIRECT_VIEW']='result'


#############################################################################
@app.route("/", methods=['GET', 'POST'])
def upload(num=None):
    if request.method == 'POST':
        f = request.files.get('file')
        f.save(os.path.join(upload_dir, f.filename))
        images = convert_from_path(os.path.join(upload_dir, f.filename))
        for i, image in enumerate(images):
            fname = "uploads/image" + str(i) + ".jpg"
            image.save(fname, "JPEG")
        # return redirect(url_for('result'))
    return render_template('homepage.html',num=num)


###############################################################################
@app.route("/result", methods=['GET', 'POST'])
def result(num=None):
    # if request.method == 'POST':
    with io.open("/content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/flask/uploads/image0.jpg",
                  'rb') as image_file:
        content = image_file.read()
    image = vision.types.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    box = 0, 0, 0, 0, 0, 0, 0, 0
    pre_box = 0, 0, 0, 0, 0, 0, 0, 0
    result_text = []
    a = ' '
    for text in texts[1:]:
        for vertex in text.bounding_poly.vertices:
            vertices = (['{},{}'.format(vertex.x, vertex.y) for vertex in text.bounding_poly.vertices])
            x1, x2, x3, x4 = vertices[0].split(',')[0], vertices[1].split(',')[0], vertices[2].split(',')[0], \
                              vertices[3].split(',')[0]
            y1, y2, y3, y4 = vertices[0].split(',')[1], vertices[1].split(',')[1], vertices[2].split(',')[1], \
                              vertices[3].split(',')[1]
            break

        box = int(x1), int(y1), int(x2), int(y2), int(x3), int(y3), int(x4), int(y4)

        if (box[1] - pre_box[5]) >= 30:
            result_text.append(a)
            a = text.description
        else:
            a += text.description + ' '
        pre_box = box
    result_text.append(a)
    print(" 🧚‍♀️ OCR running 🧚‍♀️ ")
    
    b = "\n"
    b = b.join(result_text)
    with open("/content/result_text.txt", 'w') as f:
      f.write(b)
    
    txt = ""

    with open("/content/result_text.txt") as f:
      while True:
        line = f.readline()
        txt += line
        if not line: 
          break
        txt += "\n"
    txt = [i for i in txt.split("\n") if i]

    result = spell_checker.check(txt)
    
    res = []
    for i in range(1, len(result)):
      print("spell checker", result[i][2])
      res.append(result[i][2])

    
    result = "\n".join(res)  
    result = result.replace("딸","말")

    print(" 😍  NLP running  😍 ")

    hl = Highlight(result)
    tagged = hl.highlight()
    paragraphs = [i for i in tagged.split('\n') if i]

    total = ''
    for idx in range(len(paragraphs)):
      total += "\n"
      summarizer = Summarize(result, idx)
      txt, id = summarizer.ensembleSummarize()
      print("summarized ",txt)
      paragraph = kss.split_sentences(paragraphs[idx])
      sent_counts = len(paragraph)
      for sent in range(sent_counts):
        if txt[:3] in paragraph[sent]:
          relation = ""
          string ="<u style='text-decoration:underline; text-decoration-color:#906fa8; font-weight: bold; text-decoration-style: wavy'>" + paragraph[sent] +"</u>"
          total += string
        else:
          total += paragraph[sent]
    print(total)
    with open("/content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/flask/result.txt",'w') as file:
        file.write(total)
    
    utils.save_pptx_as_png("/content/drive/'Shared drives'/'BigDATA TEAM 10'/dataCampusProject-Team10/flask/outputs", "/content/drive/'Shared drives'/'BigDATA TEAM 10'/dataCampusProject-Team10/flask/slides.pptx", overwrite_folder=True)

    with open(os.path.join(basedir, 'result.txt'), 'r', encoding="UTF-8") as file:
        string = file.readlines()
    with open(os.path.join(basedir, "relation.txt"), 'r', encoding="UTF-8") as file:
        relation = file.read()
    relation = [i for i in relation.split('\n\n') if i]
    with open(os.path.join(basedir, "templates/result_1.html"), "r", encoding="UTF-8") as file:
        result_1 = file.read()
    with open(os.path.join(basedir, "templates/result_1_1.html"), "r", encoding="UTF-8") as file:
        result_1_1 = file.read()
    with open(os.path.join(basedir, "templates/result_1_2.html"), "r", encoding="UTF-8") as file:
        result_1_2 = file.read()
    with open(os.path.join(basedir, "templates/result_1_3.html"), "r", encoding="UTF-8") as file:
        result_1_3 = file.read()
    with open(os.path.join(basedir, "templates/result_1_4.html"), "r", encoding="UTF-8") as file:
        result_1_4 = file.read()
    with open(os.path.join(basedir, "templates/result_1_5.html"), "r", encoding="UTF-8") as file:
        result_1_5 = file.read()
    with open(os.path.join(basedir, "templates/result_1_6.html"), "r", encoding="UTF-8") as file:
        result_1_6 = file.read()
    

    for j in range(1, len(string)):
        result_1 += ("<a class='nav-link waves-effect' href='#PG"+ str(j) + "'><mark><strong>Paragraph "+ str(j) + "</strong></mark></a>")         
    result_1 += ("<a class='nav-link waves-effect' href='#PP'><mark><strong>PowerPoint Slides</strong></mark></a>")

    for i in range(1, len(string)):
        result_1 += ("<div class='col-md-12' id='PG" + str(i) + "'><hr><div class='row-eq-height'><div class='col-md-12' style='line-height: 40px'><p class='dark-grey-text'><mark><strong>Paragraph" + str(i) + "</strong></mark></p><a style='line-height: 40px'>")
        result_1 += string[i] # paragraph
        result_1 += ("</a><br><div class='col-md-12 text-right'><button type='button' class='btn-yellow' data-toggle='modal' data-target='#PPT" + str(i) + "'>PPT</button><button type='button' class='btn-deep-orange' data-toggle='modal' data-target='#P" + str(i) + "'>Relation</button></div><div class='modal' id='PPT" + str(i) + "'>")
        result_1 += result_1_1
        result_1 += ("<img src='https://raw.githubusercontent.com/yoonkim313/dataCampusProject-Team10/master/5.%20Flask/slide" + str(i) + ".PNG' class='img-fluid' alt='Sample post image'>")  # ppt slide picture
        result_1 += result_1_2
        result_1 += ("<div class='modal' id='P" + str(i) + "'>")  # relation tagging modal
        result_1 += result_1_3
        result_1 += relation[i-1]  # relation tagging input
        result_1 += result_1_4
    
    result_1 += result_1_5
    result_1 += ("<div class='carousel-item active'><img src='https://raw.githubusercontent.com/yoonkim313/dataCampusProject-Team10/master/5.%20Flask/slide1.PNG' class='d-block w-100' alt='...'></div>")
    
    if len(string) > 1:
        for i in range(2, len(string)):
          result_1 += ("<div class='carousel-item'><img src='https://raw.githubusercontent.com/yoonkim313/dataCampusProject-Team10/master/5.%20Flask/slide" + str(i) + ".PNG' class='d-block w-100' alt='...'></div>")
    
    result = result_1 + result_1_6

    with open(os.path.join(basedir, "templates/final_result.html"), "w", encoding="UTF-8") as file:
        file.write(result)

    return render_template('final_result.html')
#################################################################################################################


if __name__ == '__main__':
    app.run()

/content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/flask
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://15a65d3a631c.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/Sep/2020 15:36:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2020 15:36:09] "GET /static/css/mdb.min.css HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2020 15:36:09] "GET /static/css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2020 15:36:09] "GET /static/css/style.min.css HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2020 15:36:10] "GET /static/js/jquery-3.4.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2020 15:36:10] "GET /static/js/popper.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2020 15:36:10] "GET /static/js/bootstrap.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2020 15:36:10] "GET /static/font/roboto/Roboto-Light.woff2 HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2020 15:36:10] "GET /static/js/mdb.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2020 15:36:10] "GET /static/img/svg/arrow_left.svg HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2020 15:36:10] "GET /static/img/svg/arrow_right.svg HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2020 15:36:11] "GET /static/font/roboto/Roboto-Reg

 🧚‍♀️ OCR running 🧚‍♀️ 
spell checker 평등은 자유 와 더불어 근대 사회의 핵심 이념으로 자리 잡고 있다. 인간은 가령 인종이나 성별과 상관없이 누구나 평등하다고 생각한다. 모든 인간은 평등하다고 말하는데, 이 말 은 무슨 뜻일까? 그리고 그 근거는 무엇인가? 일단 이 말을 모든 인간을 모든 측면에서 똑같이 대우하는 절대적 평등으로 생각하는 이는 없다. 인간은 저마다 다르게 가지고 태어난 능력과 소질을 똑같게 만들 수 없기 때문이다. 절대적 평등 은 개인의 개성이냐 자율성 등의 가치 와 충돌하기도 한다.
spell checker 평등에 대한 요구는 모든 불평등을 악으로 보는 것이 아니라 충분한 이유 가 제시되지 않은 불평 등 을 제거하는 데 목표를 두고 있다. ' 이유 없는 차별 금지 '라는 조건 적 평등 원칙 은 차별 대우를 할 때는 이유를 제시할 것을 요구하고 있다. 이것은 어떤 이유 가 제시된다면 특정한 부류에 속하는 사람들에게는 평등 한 대우를, 그 부류에 속하지 않는 사람들에게는 차별적 대우를 하는 것을 허용한다. 그렇다면 사람들 을 특정한 부류로 구분하는 기준 은 무엇인가? 이것은 바로 평등의 근 거에 대한 물음이다.
spell checker 근대의 여러 인권 선언에 나타난 평등 개념 은 개인 들 사이의 평등성 을 타고난 자연적 권리로 간 주하였다. 하지만 이러한 자연권 이론 은 무엇이 자연적 권리이고 권리의 존재 가 자명한 이유 가 무엇인지 등의 문제에 부딪 이게 된다. 그래서 롤스는 기존의 자연권 사상에 의존하지 않는 방식으로 인간 평등의 근거를 마른 하려고 한다. 그는 어떤 규칙 이 공평하고 일관 되게 운영되며, 그 규 칙에 따라 유사한 경우는 유사하게 취급된다면 형식적 정의는 실현된다고 본다. 하지만 롤스는 형식적 정의에 따라 규칙을 준수하는 것만으로는 정의를 담보할 수 없다고 생각한다. 그 규칙 이 더 높은 도덕적 권위를 지닌 다른 이념과 충돌할 주었기에, 실질적 정의 가 보장되기 위해서는 규칙의 내용 이 중요한

127.0.0.1 - - [12/Sep/2020 15:37:05] "GET /result HTTP/1.1" 200 -
